In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets

#Convert a PIL Image to tensor
from torchvision.transforms import ToTensor

In [ ]:
trainingData = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

testData = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batchSize = 64

trainDataloader = DataLoader(trainingData, batch_size=batchSize)
testDataloader = DataLoader(testData, batch_size=batchSize)

for X, y in testDataloader:

    #batch, channel, height, width (NCHW)
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cpu device


In [ ]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()

        '''
        input layer neurons : output layer neurons
        784 : 512
        512 : 512
        512 : 10
        '''
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        #set the gradients to zero before backpropragation as PyTorch accumulates the gradients on subsequent backward passes
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainDataloader, model, loss_fn, optimizer)
    test(testDataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.089999  [    0/60000]
loss: 2.085475  [ 6400/60000]
loss: 1.997557  [12800/60000]
loss: 2.027918  [19200/60000]
loss: 1.966590  [25600/60000]
loss: 1.888082  [32000/60000]
loss: 1.930111  [38400/60000]
loss: 1.836614  [44800/60000]
loss: 1.839817  [51200/60000]
loss: 1.775035  [57600/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 1.762335 

Epoch 2
-------------------------------
loss: 1.796193  [    0/60000]
loss: 1.779365  [ 6400/60000]
loss: 1.627076  [12800/60000]
loss: 1.692023  [19200/60000]
loss: 1.580560  [25600/60000]
loss: 1.530748  [32000/60000]
loss: 1.571722  [38400/60000]
loss: 1.477187  [44800/60000]
loss: 1.506983  [51200/60000]
loss: 1.416012  [57600/60000]
Test Error: 
 Accuracy: 61.6%, Avg loss: 1.423527 

Epoch 3
-------------------------------
loss: 1.488282  [    0/60000]
loss: 1.474119  [ 6400/60000]
loss: 1.300792  [12800/60000]
loss: 1.396857  [19200/60000]
loss: 1.278238  [25600/60000]
loss: 1.272580  [32000/600

In [ ]:

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = testData[1][0], testData[1][1]
with torch.no_grad():
    pred = model(x.to(device))
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Pullover"
